<a href="https://colab.research.google.com/github/nicks165/VectorDatabases/blob/main/Pinecone_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install "pinecone-client[grpc]"

In [ ]:
import pinecone

pinecone.init(api_key="199c7b9a-c651-4fb0-b5b9-4bc3f706b9b3",
              environment="us-east-1-aws")


In [ ]:
active_indexes = pinecone.list_indexes()
print(active_indexes)

In [ ]:
!pip install -qU pinecone-client[grpc] datasets sentence-transformers

In [ ]:
from datasets import load_dataset

#dataset = load_dataset('quora', split='train')
#dataset
docs = load_dataset(f"Cohere/wikipedia-22-12-zh-embeddings", split="train")
#docs

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print(f"You are using {device}. This is much slower than using "
          "a CUDA-enabled GPU. If on Colab you can change this by "
          "clicking Runtime > Change runtime type > GPU.")

model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

In [ ]:
index_name = 'semantic-search-fast1'

# only create index if it doesn't exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=model.get_sentence_embedding_dimension(),
        metric='cosine'
    )

# now connect to the index
index = pinecone.GRPCIndex(index_name)

In [ ]:
import time
batch_size = 100
#print(len(questions))
#size=len(questions)
e_size=len(questions)
i=0
i_end = 100
# create IDs batch
ids = [str(x) for x in range(i, i_end)]
# create metadata batch
metadatas = [{'text': text} for text in questions[i:i_end]]
# create embeddings
xc = model.encode(questions[i:i_end])
# create records list for upsert
records = zip(ids, xc, metadatas)
start_time = time.time()    
# upsert to Pinecone
index.upsert(vectors=records)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
from IPython.utils.text import string

import time

# add 1 more record
eText1 = "How is the meetha?"
#metadata1 = [{'text': text} for text in [eText1]]
#codedText1 = model.encode([eText1])

# add 1 more record
eText2 = "# 2 How is the meetha?"
#metadata2 = [{'text': text} for text in [eText2]]
#codedText2 = model.encode([eText2])

ewMetadata = [{'text': text} for text in [eText1, eText2]]
codedText = model.encode([eText1, eText2])
ewRecords = zip([str(hash(eText1)), str(hash(eText2))], 
              codedText,
              ewMetadata
              )

start_time = time.time() 
#index.upsert([(str(hash(eText1))[1:10], codedText1, metadata1),
#              (str(hash(eText2))[1:10], codedText2, metadata2)])

index.upsert(vectors=ewRecords)

print("--- %s seconds ---" % (time.time() - start_time))

# check number of records in the index
index.describe_index_stats()

In [ ]:
query = "Where is mount everest?"
import time

# create the query vector
xq = model.encode(query).tolist()

# now query
start_time = time.time()
xc = index.query(xq, top_k=3, include_metadata=True)
print("--- %s seconds ---" % (time.time() - start_time))
xc
for result in xc['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")


Cohere example

In [1]:
!pip install -U cohere pinecone-client datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━

In [6]:
from datasets import load_dataset
import torch
import cohere




#Load all documents + embeddings
limit = -1 # keep -1 for all, else update to a positive number to limit

max_docs_loaded = 0
docs_stream = load_dataset(f"Cohere/wikipedia-22-12-zh-embeddings", split="train", streaming=True)

docs = []
doc_embeddings = []

for doc in docs_stream:
    docs.append(doc)
    doc_embeddings.append(doc['emb'])
    max_docs_loaded += 1
    if (limit > 0 and max_docs_loaded == limit):
      break

In [7]:
import numpy as np

shape = np.array(doc_embeddings).shape
print(shape)

(1, 768)


In [8]:
import pinecone

pinecone.init(api_key="199c7b9a-c651-4fb0-b5b9-4bc3f706b9b3",
              environment="us-east-1-aws")


In [10]:
index_name = 'cohere-wiki-2m'
dimension = shape[1]
# only create index if it doesn't exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=dimension,
        metric='cosine'
    )

# now connect to the index
index = pinecone.GRPCIndex(index_name)

ApiException: ignored

In [ ]:
import time
batch_size = 100

ids = [str(i) for i in range(shape[0])]

# create list of metadata dictionaries
meta = [{'text': text} for text in docs]

# create list of (id, vector, metadata) tuples to be upserted
to_upsert = list(zip(ids, doc_embeddings, meta))

start_time = time.time()
for i in range(0, shape[0], batch_size):
    i_end = min(i+batch_size, shape[0])
    index.upsert(vectors=to_upsert[i:i_end])

    if(i_end in [1000, 10000, 100000, 500000, 1000000, 2000000]):
      print(" For {0} entries, time taken for inserts = {1} ".format(i_end, time.time() - start_time))


# let's view the index statistics
print(index.describe_index_stats())

In [ ]:
co = cohere.Client(f"<<COHERE_API_KEY>>")  # Add your cohere API key from www.cohere.com

query = "What was the cause of the major recession in the early 20th century?"

# create the query embedding
xq = co.embed(texts=[query], model='multilingual-22-12').embedding

# query, returning the top 5 most similar results
res = index.query(xq, top_k=5, include_metadata=True)

for match in res['matches']:
    print(f"{match['score']:.2f}: {match['metadata']['text']}")

**Do not run this**

In [ ]:
from datasets import load_dataset
import torch
import cohere


co = cohere.Client(f"<<COHERE_API_KEY>>")  # Add your cohere API key from www.cohere.com

#Load all documents + embeddings
max_docs_loaded = 0
docs_stream = load_dataset(f"Cohere/wikipedia-22-12-zh-embeddings", split="train", streaming=True)

docs = []
doc_embeddings = []

for doc in docs_stream:
    docs.append(doc)
    doc_embeddings.append(doc['emb'])
    max_docs_loaded++

#doc_embeddings = torch.tensor(doc_embeddings)

query = 'Who founded Youtube'
response = co.embed(texts=[query], model='multilingual-22-12')
query_embedding = response.embeddings 
query_embedding = torch.tensor(query_embedding)

# Compute dot score between query embedding and document embeddings
dot_scores = torch.mm(query_embedding, doc_embeddings.transpose(0, 1))
top_k = torch.topk(dot_scores, k=3)

# Print results
print("Query:", query)
for doc_id in top_k.indices[0].tolist():
    print(docs[doc_id]['title'])
    print(docs[doc_id]['text'], "\n")